# 誤差逆伝播法

## 数値微分の難点

前回までで、ニューラルネットワークの基本的な学習方法である、確率的勾配降下法は実装できた

このとき、勾配の計算は数値微分によって実装していた

数値微分はシンプルに実装できて分かりやすいが、計算に時間がかかるという難点がある

そこで、ここでは、重みパラメータの勾配計算を効率良く行う**誤差逆伝播法**を実装する


## 計算グラフ

誤差逆伝播法を視覚的に理解する方法として**計算グラフ（Computational graph）**というものがある

計算グラフとは計算の過程をグラフによって表したものである

ここで言うグラフとは、データ構造としてのグラフであり、複数のノードとエッジ（ノード間を結ぶ直線）によって表現されるものである

### 計算グラフで計算問題を解く
以下のような簡単な計算問題を、計算グラフを使って解いてみる

**問:** スーパーで1個100円のりんごを2個買ったとき、支払う金額はいくらか？（消費税は10％とする）

計算グラフは以下のように、ノードごとの計算結果が左から右へ伝わるように表現する

![computational_graph01.png](./img/computational_graph01.png)

上記では、「×2」や「×1.1」を一つの演算として○（ノード）でくくっているが、「×」のような演算子は単一のノードで表現するのが望ましい

この場合、「2」と「1.1」は、それぞれ「りんごの個数」と「消費税」という変数として、ノードの外側に表記する

![computational_graph02.png](./img/computational_graph02.png)

次に、もう少し複雑な計算を計算グラフで解く

**問:** スーパーでりんご（100円／個）を2個、みかん（150円／個）を3個買ったとき、支払う金額はいくらか？（消費税は10％とする）

この問題を解くための計算グラフは以下のようになる

![computational_graph03.png](./img/computational_graph03.png)

このように、計算グラフを使って問題を解くには、

1. 計算式に使われる要素を演算子と変数に分ける
2. 演算子と変数をノードとしてエッジでつなぐ
3. 計算グラフ上で計算を左から右へ進める

という流れで作業する

ここで、「計算を左から右へ進める」処理を**順伝播（forward propagation）**と呼び、ニューラルネットワークの**推論処理**に対応する

逆に「計算を右から左へ戻る」処理を**逆伝播（back propagation）**と呼び、ニューラルネットワークの**学習処理**に対応する

この逆伝播は、この先、微分を計算するにあたって重要な働きをする

### 局所的計算と逆伝播による微分
計算グラフを使う利点は大きく以下の2点がある

- 「局所的な計算」を伝播することにより複雑な計算を行うことができる
    - 各ノードは、全体の計算には関与せず「自分に関係する小さな範囲」の計算だけを行う（局所的計算）
    - これにより計算を単純化することができる
    - また、計算途中の結果をすべて、各ノードで保持することも可能
- 計算グラフを逆伝播することで微分値を効率よく計算できる
    - 順伝播が「局所的な計算」であるのと同様に、逆伝播は「局所的な微分」を表す
    - これにより微分計算を単純化し、計算速度を向上させることができる

例えば上記の問題について、りんごの値段が値上がりした場合、最終的な支払金額にどのように影響するか知りたいとする

これは「りんごの値段に対する支払金額の微分」を求めることに相当する（りんごの値段を $x$, 支払金額を $L$ とした場合、$∂L/∂x$ を計算することに相当する）

計算グラフの逆伝播によって、この問題を解くと以下のようになる

![computational_graph04.png](./img/computational_graph04.png)

上記のように、計算グラフを右から左へ計算することで「局所的な微分」を伝達することができる

この結果から「りんごの値段に関する支払金額の微分」の値は 2.2 であると言える

すなわち、りんごが1円値上がりしたら、最終的な支払金額は2.2円増えることを意味する（正確には、りんごの値段がある微小な値だけ増えたら、最終的な支払金額はその微小な値の2.2倍だけ増加することを意味する）

## 連鎖律

### 計算グラフの逆伝播
計算グラフの逆伝播を一般化すると以下のように表すことができる

![computational_graph05.png](./img/computational_graph05.png)

ここで、上記計算グラフは $y = f(x)$ という計算を表現している

逆伝播の計算手順は、信号 $E$ に対して、ノードの局所的な微分 $\frac{∂y}{∂x}$ を乗算し、次のノードで伝達する、というものになっている

これは、前述したりんごの支払金額計算で考えると分かりやすい

```
最初のノード: f(x) = 2x (x: 前のノードの出力値＝りんごの値段, 2: りんごの個数)
最後のノード: g(x) = 1.1x (x: 前のノードの出力値, 1.1: 消費税倍率)

最後のノードの逆伝播: 1 * g'(x) = 1 * 1.1 = 1.1
最初のノードの逆伝播: 1.1 * f'(x) = 1.1 * 2 = 2.2
```

この計算により効率よく微分値を求めることができるだが、その理由は**連鎖律の原理**から説明できる

### 連鎖律
以下のような計算グラフを考える

![computational_graph06.png](./img/computational_graph06.png)

この計算を微分すると、以下のような逆伝播で表現される

![computational_graph07.png](./img/computational_graph07.png)

ここで、合成関数の定理より $\frac{∂z}{∂z} \frac{∂z}{∂y}$ の $∂z$ は "打ち消し合い"、$\frac{∂z}{∂y}$ となる

同様にして $\frac{∂z}{∂z} \frac{∂z}{∂y} \frac{∂y}{∂x} = \frac{∂z}{∂x}$ となる

このような合成関数の微分の性質を**連鎖律**と呼ぶ

連鎖律により、計算の一部を "打ち消す" ことができるため、効率よく微分計算ができるという仕組みである

## 単純な算術ノードの実装

計算グラフの単純なノードを実装していく

ただし、型名は「ノード」ではなく、ニューラルネットワークの「層（レイヤ）」を意味するものとして `***Layer` という名前で実装することにする

In [1]:
# すべてのレイヤの基底となる抽象型: 抽象レイヤ
abstract type AbstractLayer end

### 加算ノード（加算レイヤ）
まず、加算ノード $z = x + y$ について考える

このノードの逆伝播（偏微分）は以下のようになる

$$ \begin{array}{ll}
    \frac{∂z}{∂x} = 1 \\
    \frac{∂z}{∂y} = 1
\end{array} $$

従って、この計算グラフは以下のようになる

![computational_graph_add.png](./img/computational_graph_add.png)

上図のように、加算ノードの逆伝播は、上流の値がそのまま分岐して流れていく

これを実装すると以下のようになる

In [2]:
# 加算レイヤ
mutable struct AddLayer <: AbstractLayer end

# 加算レイヤ: 順伝播
## x, y: 上流から流れてきる2値 => 下流に流す値
forward(layer::AddLayer, x::Float64, y::Float64)::Float64 = x + y

# 加算レイヤ: 逆伝播
## dout: 上流から流れてくる微分値 => 下流に流す2値
backward(layer::AddLayer, dout::Float64)::Tuple{Float64,Float64} = (dout * 1, dout * 1)

backward (generic function with 1 method)

### 乗算ノード（乗算レイヤ）
同様に、乗算ノード $z = x \times y$ について考えると、このノードの逆伝播（偏微分）は以下のようになる

$$ \begin{array}{ll}
    \frac{∂z}{∂x} = y \\
    \frac{∂z}{∂y} = x
\end{array} $$
 
従って、この計算グラフは以下のようになる

![computational_graph_mul.png](./img/computational_graph_mul.png)

すなわち、乗算ノードの逆伝播では、上流から流れてきた微分値に対して、順伝播の "ひっくり返した値" を乗算して流す形になる

これを実装すると以下のようになる

In [3]:
# 乗算レイヤ
mutable struct MulLayer <: AbstractLayer
    x::Float64
    y::Float64
end

MulLayer() = MulLayer(0, 0)

# 乗算レイヤ: 順伝播
## x, y: 上流から流れてきる2値 => 下流に流す値
forward(layer::MulLayer, x::Float64, y::Float64)::Float64 = begin
    # 順伝播時に値を保持しておく
    layer.x = x
    layer.y = y
    x * y
end

# 乗算レイヤ: 逆伝播
## dout: 上流から流れてくる微分値 => 下流に流す2値
backward(layer::MulLayer, dout::Float64)::Tuple{Float64,Float64} = (dout * layer.y, dout * layer.x)

backward (generic function with 2 methods)

これで加算レイヤと乗算レイヤを実装できたため、これらを用いて少し複雑な計算グラフを実装する

前述した「りんご2個とみかん3個の買い物」の計算グラフを以下に示す

![computational_graph_backword.png](./img/computational_graph_backword.png)

これを実装すると以下のようになる

In [6]:
# 変数
apple = 100.0
apple_num = 2.0
orange = 150.0
orange_num = 3.0
tax = 1.1

# レイヤ（ノード）
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 順伝播
apple_price = forward(mul_apple_layer, apple, apple_num)
orange_price = forward(mul_orange_layer, orange, orange_num)
all_price = forward(add_apple_orange_layer, apple_price, orange_price)
price = forward(mul_tax_layer, all_price, tax) # => 715

715.0000000000001

In [8]:
# 逆伝播
d_price = 1.0
d_all_price, d_tax = backward(mul_tax_layer, d_price)
d_apple_price, d_orange_price = backward(add_apple_orange_layer, d_all_price)
d_orange, d_orange_num = backward(mul_orange_layer, d_orange_price)
d_apple, d_apple_num = backward(mul_apple_layer, d_apple_price)

println("$d_apple_num, $d_apple, $d_orange_num, $d_orange, $d_tax")
# => d_apple_num: 110, d_apple: 2.2, d_orange_num: 165, d_orange: 3.3, d_tax: 650

110.00000000000001, 2.2, 165.0, 3.3000000000000003, 650.0
